# 🌐 Problem: Maintaining Structural Context for Data-Rich Documents

This notebook demonstrates how to extract structured context information from **HTML documents** using two powerful tools:

- **Unstructured**: Extracts elements (text, titles, tables, images) preserving document structure.
- **SparkNLP (Reader2Table)**: Reads HTML files into Spark NLP pipelines as text documents for large-scale NLP tasks.

In [2]:
!pip install unstructured[local-inference]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.1/529.1 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 95.2 MB/s eta 0:00:00
   

In [3]:
!wget -P html_docs !wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/SPARKNLP-1320-Benchmarkunstructured-vs-SparkNLP-in-unstructured-document-processing/open-source-nlp/data/readers/html/EHR-2025-12-000002.html

--2025-12-26 20:46:37--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2025-12-26 20:46:37--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/SPARKNLP-1320-Benchmarkunstructured-vs-SparkNLP-in-unstructured-document-processing/open-source-nlp/data/readers/html/EHR-2025-12-000002.html
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16690 (16K) [text/plain]
Saving to: ‘html_docs/EHR-2025-12-000002.html’

EHR-2025-12-000002. 100%[===================>]  16.30K  --.-KB/s    in 0s      

2025-12-26 20:46:37 (41.7 MB/s) - ‘html_docs/EHR-2025-12-000002.html’ saved [16690/16690]

FINISHED --2025-12-26 20:46:37--
Total wall clock time: 0.2s
Downloaded: 1 files, 16K in 0s (41.

## Unstructured: HTML Partition and Cleaning

In [5]:
from unstructured.partition.html import partition_html
from unstructured.cleaners.core import (
    clean_extra_whitespace,
    replace_unicode_quotes,
    clean_non_ascii_chars,
    clean_bullets
)
import json
import os
import re


def remove_colons(text: str) -> str:
    return re.sub(r":", "", text)

def clean_element_text(text: str) -> str:
    """Clean up text extracted from HTML using Unstructured."""
    text = clean_extra_whitespace(text)
    text = replace_unicode_quotes(text)
    text = clean_non_ascii_chars(text)
    text = clean_bullets(text)
    text = remove_colons(text)
    return text.strip()

def ingest_and_clean_unstructured_html_tables(html_path: str):
    """
    Extract and clean only HTML table data using Unstructured.
    Returns a simplified JSON list of tables and their contents.
    """
    elements = partition_html(filename=html_path)
    cleaned_output = []

    for el in elements:
        # Filter only 'Table' elements
        if hasattr(el, 'category') and el.category == 'Table':
            # Try to extract structured table data if available
            table_content = getattr(el, 'text', None)
            if table_content:
                cleaned_text = clean_element_text(table_content)
                cleaned_output.append({
                    "filename": os.path.basename(html_path),
                    "type": "Table",
                    "text": cleaned_text
                })
    return cleaned_output

In [8]:
file_path = 'html_docs/EHR-2025-12-000002.html'
output_html = ingest_and_clean_unstructured_html_tables(file_path)

In [9]:
output_html

[{'filename': 'EHR-2025-12-000002.html',
  'type': 'Table',
  'text': 'Test Result Units Reference Range Status PSA 0.32 ng/mL 0-4.0 Excellent Testosterone 125 ng/dL 300-1000 Recovering Hemoglobin 14.3 g/dL 13.5-17.5 Normal WBC 7.2 K/uL 4.5-11.0 Normal Creatinine 0.9 mg/dL 0.7-1.3 Normal ALT 22 U/L 7-56 Normal'},
 {'filename': 'EHR-2025-12-000002.html',
  'type': 'Table',
  'text': 'Test Result Units Reference Range Status Testosterone 105 ng/dL 300-1000 Recovering Hemoglobin 12.3 g/dL 13.5-17.5 Normal Creatinine 0.7 mg/dL 0.7-1.3 Normal'},
 {'filename': 'EHR-2025-12-000002.html',
  'type': 'Table',
  'text': 'Medication Dose Frequency Indication Status Atorvastatin (Lipitor) 10 mg PO Daily Hyperlipidemia Active Aspirin 81 mg PO Daily Cardiovascular prophylaxis Active Vitamin D3 2000 IU PO Daily Bone health Active Calcium carbonate 500 mg PO BID Bone health (post-ADT) Active Multivitamin 1 tab PO Daily Nutritional support Active'}]

 ## SparkNLP: Reader2Table for HTML

In [10]:
!pip install -q --force-reinstall pyspark==3.5.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 13.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 1.0.1 requires pyspark[connect]~=4.0.0, but you have pyspark 3.5.7 which is incompatible.


In [11]:
 !pip install spark-nlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.0/745.0 kB 18.6 MB/s eta 0:00:00


In [12]:
import sparknlp
from sparknlp.reader.reader2table import Reader2Table
from sparknlp.annotator import DocumentNormalizer
from pyspark.ml import Pipeline

spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.5.7


In [14]:
print("Spark version:", spark.version)
print("Scala Version:", spark.sparkContext._jvm.scala.util.Properties.versionString())
print("Java Version:", spark.sparkContext._jvm.java.lang.System.getProperty("java.version"))

Spark version: 3.5.7
Scala Version: version 2.12.18
Java Version: 17.0.17


In [15]:
empty_df = spark.createDataFrame([], 'string').toDF('text')

reader2doc = Reader2Table() \
    .setContentType('text/html') \
    .setContentPath('html_docs/EHR-2025-12-000002.html') \
    .setOutputCol('table')

pipeline = Pipeline(stages=[reader2doc])
model = pipeline.fit(empty_df)
result_df = model.transform(empty_df)

# Display the exploded DataFrame
result_df.show(truncate=False, n=5)

+-----------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
from pyspark.sql.functions import explode, col

result_df.select(explode("table")).show(truncate=False, n=5)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|col                                                                                                                                                                                                                                                               

In [17]:
result_df.select(explode("table.metadata")).show(truncate=False, n=5)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|col                                                                                                                                                                                                                                                                                                            |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{element_id -> 2656cd3d-ef11-47d0-91b3-122b199f07a6, parent_id -> 26af5c79-8246-4